# Finetuning LLM using QLora on English Poems Dataset

Some of the important references I used for writing this notebook -

i) https://colab.research.google.com/drive/1Vvju5kOyBsDr7RX_YAvp6ZsSOoSMjhKD?usp=sharing#scrollTo=E0Nl5mWL0k2T,

ii) https://colab.research.google.com/drive/1iERDk94Jp0UErsPf7vXyPKeiM4ZJUQ-a?usp=sharing,

iii) https://huggingface.co/datasets/biglam/gutenberg-poetry-corpus

## Setting up our LLM model

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

We use the gpt-neox-20b model.

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neox-20b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model-00043-of-00046.safetensors:   0%|          | 0.00/910M [00:00<?, ?B/s]

model-00044-of-00046.safetensors:   0%|          | 0.00/910M [00:00<?, ?B/s]

model-00045-of-00046.safetensors:   0%|          | 0.00/604M [00:00<?, ?B/s]

model-00046-of-00046.safetensors:   0%|          | 0.00/620M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/46 [00:00<?, ?it/s]

In [3]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8650752 || all params: 10597552128 || trainable%: 0.08162971878329976


We are training only 0.08 % if the weights.

## Loading the Dataset

We now load Gutenberg-Poetry-Corpus, perhaps the most exhaustive English poems dataset till date.

In [7]:
from datasets import load_dataset
data = load_dataset("biglam/gutenberg-poetry-corpus")

In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['line', 'gutenberg_id'],
        num_rows: 3085117
    })
})

In [13]:
data = data.map(lambda samples: tokenizer(samples["line"]), batched=True)

Map:   0%|          | 0/3085117 [00:00<?, ? examples/s]

## Output before training

In [18]:
device = "cuda:0"
from IPython.display import display, Markdown

def make_inference(text):
  batch = tokenizer(text, return_tensors='pt').to(device)

  output_tokens = model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens= True))))

Let's see the output before training the model.

In [21]:
make_inference("Write a poem.")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Write a poem.

Write a story.

Write a letter.

Write a letter to a friend.

Write a letter to a friend about a problem.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.

Write a letter to a friend about a problem and send it.



Ummm....can't really call that a poem :)

## Training

We train the model for 10 steps.

In [23]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.955700
2,6.004900
3,4.680200
4,4.798700
5,4.176400
6,5.071600
7,5.168900
8,4.610300
9,5.432900
10,5.036900


TrainOutput(global_step=10, training_loss=4.993649053573608, metrics={'train_runtime': 159.5483, 'train_samples_per_second': 0.251, 'train_steps_per_second': 0.063, 'total_flos': 49337195249664.0, 'train_loss': 4.993649053573608, 'epoch': 1.296547262227008e-05})

## Output after training

Now we test the output for the same question.

In [24]:
make_inference("Write a poem.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1354: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.1

Write a poem.

|

And if thou wilt, a tale of love,

|

To while away the night,

|

And with thy voice to charm mine ear,

|

And with thy looks to cheer mine eye.

|

And if thou wilt, a tale of woe,

|

To while away the day,

|

And with thy voice to soothe mine ear,

|

And with thy looks to cheer mine eye.

|

And if thou wilt, a tale of death,

|

To while away the night,

|

And with thy voice to charm mine ear,

|

And with thy looks to cheer mine eye.

|

And if thou wilt, a tale of joy,

|

To while away the day,

|

And

Not a bad poem!
Of course, we can do much better if we train for many more steps, we have done it for just 10 so far.